## Install relevant packages

In [1]:
%%capture

!pip install unsloth # install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git # Also get the latest version Unsloth!

## Import all relevant packages

In [2]:
# Modules for fine-tuning
from unsloth import FastLanguageModel
import torch # Import PyTorch
from trl import SFTTrainer # Trainer for supervised fine-tuning (SFT)
from unsloth import is_bfloat16_supported # Checks if the hardware supports bfloat16 precision
# Hugging Face modules
from huggingface_hub import login # Lets you login to API
from transformers import TrainingArguments # Defines training hyperparameters
from datasets import load_dataset # Lets you load fine-tuning datasets
# Import weights and biases
import wandb
# Import kaggle secrets
from kaggle_secrets import UserSecretsClient

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


## Create API keys and login to Hugging Face and Weights and Biases

In [3]:
# Initialize Hugging Face & WnB tokens
user_secrets = UserSecretsClient() # from kaggle_secrets import UserSecretsClient
hugging_face_token = user_secrets.get_secret("HF_TOKEN")
wnb_token = user_secrets.get_secret("WB_TOKEN")

# Login to Hugging Face
login(hugging_face_token) # from huggingface_hub import login

# Login to WnB
wandb.login(key=wnb_token) # import wandb
run = wandb.init(
    project='Fine-tune-DeepSeek-R1-Distill-Llama-8B on Industrial assets maintenance', 
    job_type="training", 
    anonymous="allow"
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: assia-chafii93 (assia-chafii93-insa-lyon). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250201_234456-b0gix6si
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run deft-hill-2
wandb: ⭐️ View project at https://wandb.ai/assia-chafii93-insa-lyon/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%20Industrial%20assets%20maintenance
wandb: 🚀 View run at https://wandb.ai/assia-chafii93-insa-lyon/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%20Industrial%

## Loading DeepSeek R1 and the Tokenizer


**What are we doing in this step?**

In this step, we **load the DeepSeek R1 model and its tokenizer** using `FastLanguageModel.from_pretrained()`. We also **configure key parameters** for efficient inference and fine-tuning. We will be using a distilled 8B version of R1 for faster computation.  

**Key parameters explained**
```py
max_seq_length = 2048  # Define the maximum sequence length a model can handle (i.e., number of tokens per input)
dtype = None  # Default data type (usually auto-detected)
load_in_4bit = True  # Enables 4-bit quantization – a memory-saving optimization
```

**Intuition behind 4-bit quantization**

Imagine compressing a **high-resolution image** to a smaller size—**it takes up less space but still looks good enough**. Similarly, **4-bit quantization reduces the precision of model weights**, making the model **smaller and faster while keeping most of its accuracy**. Instead of storing precise **32-bit or 16-bit numbers**, we compress them into **4-bit values**. This allows **large language models to run efficiently on consumer GPUs** without needing massive amounts of memory. 

In [4]:
# Set parameters
max_seq_length = 2048 # Define the maximum sequence length a model can handle (i.e. how many tokens can be processed at once)
dtype = None # Set to default 
load_in_4bit = True # Enables 4 bit quantization — a memory saving optimization 

# Load the DeepSeek R1 model and tokenizer using unsloth — imported using: from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # Load the pre-trained DeepSeek R1 model (8B parameter version)
    max_seq_length=max_seq_length, # Ensure the model can process up to 2048 tokens at once
    dtype=dtype, # Use the default data type (e.g., FP16 or BF16 depending on hardware support)
    load_in_4bit=load_in_4bit, # Load the model in 4-bit quantization to save memory
    token=hugging_face_token, # Use hugging face token
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.48.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/231 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.9k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

## Testing DeepSeek R1 on a industrial maintenance use-case before fine-tuning
### Defining a system prompt¶
To create a prompt style for the model, we will define a system prompt and include placeholders for the question and response generation. The prompt will guide the model to think step-by-step and provide a logical, accurate response.

In [5]:
prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an industrial maintenance expert with advanced knowledge in pump motors, controllers, diagnostics, and troubleshooting. 
Please answer the following maintenance-related question.

### Question:
{}

### Response:
<think>{}"""

### Running inference on the model

In this step, we **test the DeepSeek R1 model** by providing a **industrial maintenance question** and generating a response.  
The process involves the following steps:

1. **Define a test question** related to a industrial maintenance case.
2. **Format the question using the structured prompt (`prompt_style`)** to ensure the model follows a logical reasoning process.
3. **Tokenize the input and move it to the GPU (`cuda`)** for faster inference.
4. **Generate a response using the model**, specifying key parameters like `max_new_tokens=1200` (limits response length).
5. **Decode the output tokens back into text** to obtain the final readable answer.

In [6]:
# Creating a industrial maintenance question for inference
question = """A centrifugal pump motor is drawing 15% higher current than rated, vibrating excessively, emitting a burning smell, and running 10°C hotter than normal. The discharge pressure has also dropped by 10%. What could be the root cause, and what steps should be taken to resolve the issue?"""

# Enable optimized inference mode for Unsloth models (improves speed and efficiency)
FastLanguageModel.for_inference(model)  # Unsloth has 2x faster inference!

# Format the question using the structured prompt (`prompt_style`) and tokenize it
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")  # Convert input to PyTorch tensor & move to GPU

# Generate a response using the model
outputs = model.generate(
    input_ids=inputs.input_ids, # Tokenized input question
    attention_mask=inputs.attention_mask, # Attention mask to handle padding
    max_new_tokens=1200, # Limit response length to 1200 tokens (to prevent excessive output)
    use_cache=True, # Enable caching for faster inference
)

# Decode the generated output tokens into human-readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the relevant response part (after "### Response:")
print(response[0].split("### Response:")[1])  


<think>
Okay, so I'm trying to figure out what's causing this centrifugal pump motor issue. Let me go through the symptoms one by one and see what they could mean.

First, the motor is drawing 15% higher current than rated. That's a significant increase, more than just a minor overload. High current could be due to a few things. Maybe the voltage is too high, which would cause the motor to work harder than it's supposed to. Or perhaps there's a problem with the windings or the rotor, causing it to require more current. I need to consider both electrical and mechanical issues here.

Next, the motor is vibrating excessively. Excessive vibration can be a sign of loose parts, imbalance, or wear and tear. If the housing isn't properly mounted or if there's an issue with the bearings, this could happen. It could also be a symptom of a bad rotor or stator, but I'm not sure yet.

Then, there's the burning smell. That's worrying. A burning smell usually indicates some kind of electrical issue,

>**Before starting fine-tuning — why are we fine-tuning in the first place?**
>
> Even without fine-tuning, our model successfully generated a chain of thought and provided reasoning before delivering the final answer. The reasoning process is encapsulated within the `<think>` `</think>` tags. So, why do we still need fine-tuning? The reasoning process, while detailed, was long-winded and not concise. Additionally, we want the final answer to be consistent in a certain style. 



## Fine-tuning step by step

### Step 1 — Update the system prompt 
We will slightly change the prompt style for processing the dataset by adding the third placeholder for the complex chain of thought column. `</think>`

In [7]:
# Updated training prompt style to add </think> tag 
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context. 
Write a response that appropriately completes the request. 
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
You are an industrial maintenance expert with advanced knowledge in pump motors, controllers, diagnostics, and troubleshooting. 
Please answer the following maintenance-related question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""


### Step 2 — Download the fine-tuning dataset and format it for fine-tuning

We will use a generated dataset

In [8]:
dataset = load_dataset("/kaggle/input/indus-asset", split = "train[0:500]",trust_remote_code=True) # Keep only first 500 rows
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['problem_statement', 'solution_reasoning', 'final_answer'],
    num_rows: 30
})

In [9]:
dataset[1]

{'problem_statement': 'A variable frequency drive (VFD) controlling a pump exhibits erratic speed variations. What could be the root cause?',
 'solution_reasoning': 'A VFD controls motor speed by adjusting frequency and voltage. (1) Electrical noise can cause erratic behavior—inspect grounding and shielding; (2) Sensor feedback (e.g., pressure or flow sensors) may provide unstable input—verify sensor calibration; (3) Software parameters, such as PID tuning, may be incorrect—check settings and logs; (4) Internal VFD faults or overheating could trigger protective cycling—inspect error codes and thermal status.',
 'final_answer': 'Check sensor stability, grounding/shielding for electrical noise, and PID control settings in the VFD.'}

>**Next step is to structure the fine-tuning dataset according to train prompt style—why?**
>
> - Each question is paired with chain-of-thought reasoning and the final response.
> - Ensures every training example follows a consistent pattern.
> - Prevents the model from continuing beyond the expected response lengt by adding the EOS token.

In [10]:
# We need to format the dataset to fit our prompt training style 
EOS_TOKEN = tokenizer.eos_token  # Define EOS_TOKEN which the model when to stop generating text during training
EOS_TOKEN

'<｜end▁of▁sentence｜>'

In [11]:
# Define formatting prompt function
def formatting_prompts_func(examples):  # Takes a batch of dataset examples as input
    inputs = examples["problem_statement"]       # Extracts the medical question from the dataset
    cots = examples["solution_reasoning"]      # Extracts the chain-of-thought reasoning (logical step-by-step explanation)
    outputs = examples["final_answer"]      # Extracts the final model-generated response (answer)
    
    texts = []  # Initializes an empty list to store the formatted prompts
    
    # Iterate over the dataset, formatting each question, reasoning step, and response
    for input, cot, output in zip(inputs, cots, outputs):  
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN  # Insert values into prompt template & append EOS token
        texts.append(text)  # Add the formatted text to the list

    return {
        "text": texts,  # Return the newly formatted dataset with a "text" column containing structured prompts
    }

In [12]:
# Update dataset formatting
dataset_finetune = dataset.map(formatting_prompts_func, batched = True)
dataset_finetune["text"][0]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

'Below is an instruction that describes a task, paired with an input that provides further context. \nWrite a response that appropriately completes the request. \nBefore answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.\n\n### Instruction:\nYou are an industrial maintenance expert with advanced knowledge in pump motors, controllers, diagnostics, and troubleshooting. \nPlease answer the following maintenance-related question.\n\n### Question:\nA centrifugal pump motor is overheating during operation despite normal load conditions. What could be the cause?\n\n### Response:\n<think>\nTo diagnose the issue, we consider multiple factors: (1) Verify voltage supply and phase imbalance; (2) Check for excessive friction due to misalignment or worn bearings; (3) Inspect cooling systems (e.g., clogged ventilation or fan failure); (4) Evaluate insulation resistance to detect short circuits. Since the load is normal,

### Step 3 — Setting up the model using LoRA

**An intuitive explanation of LoRA** 

Large language models (LLMs) have **millions or even billions of weights** that determine how they process and generate text. When fine-tuning a model, we usually update all these weights, which **requires massive computational resources and memory**.

LoRA (**Low-Rank Adaptation**) allows to fine-tune efficiently by:

- Instead of modifying all weights, **LoRA adds small, trainable adapters** to specific layers.  
- These adapters **capture task-specific knowledge** while leaving the original model unchanged.  
- This reduces the number of trainable parameters **by more than 90%**, making fine-tuning **faster and more memory-efficient**.  

Think of an LLM as a **complex factory**. Instead of rebuilding the entire factory to produce a new product, LoRA **adds small, specialized tools** to existing machines. This allows the factory to adapt quickly **without disrupting its core structure**.

For a more technical explanation, check out this tutorial by [Sebastian Raschka](https://www.youtube.com/watch?v=rgmJep4Sb4&t).

Below, we will use the `get_peft_model()` function which stands for Parameter-Efficient Fine-Tuning — this function wraps the base model (`model`) with LoRA modifications, ensuring that only specific parameters are trained.

In [13]:
# Apply LoRA (Low-Rank Adaptation) fine-tuning to the model 
model_lora = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank: Determines the size of the trainable adapters (higher = more parameters, lower = more efficiency)
    target_modules=[  # List of transformer layers where LoRA adapters will be applied
        "q_proj",   # Query projection in the self-attention mechanism
        "k_proj",   # Key projection in the self-attention mechanism
        "v_proj",   # Value projection in the self-attention mechanism
        "o_proj",   # Output projection from the attention layer
        "gate_proj",  # Used in feed-forward layers (MLP)
        "up_proj",    # Part of the transformer’s feed-forward network (FFN)
        "down_proj",  # Another part of the transformer’s FFN
    ],
    lora_alpha=16,  # Scaling factor for LoRA updates (higher values allow more influence from LoRA layers)
    lora_dropout=0,  # Dropout rate for LoRA layers (0 means no dropout, full retention of information)
    bias="none",  # Specifies whether LoRA layers should learn bias terms (setting to "none" saves memory)
    use_gradient_checkpointing="unsloth",  # Saves memory by recomputing activations instead of storing them (recommended for long-context fine-tuning)
    random_state=3407,  # Sets a seed for reproducibility, ensuring the same fine-tuning behavior across runs
    use_rslora=False,  # Whether to use Rank-Stabilized LoRA (disabled here, meaning fixed-rank LoRA is used)
    loftq_config=None,  # Low-bit Fine-Tuning Quantization (LoFTQ) is disabled in this configuration
)

Unsloth 2025.1.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [14]:
# Initialize the fine-tuning trainer — Imported using from trl import SFTTrainer
trainer = SFTTrainer(
    model=model_lora,  # The model to be fine-tuned
    tokenizer=tokenizer,  # Tokenizer to process text inputs
    train_dataset=dataset_finetune,  # Dataset used for training
    dataset_text_field="text",  # Specifies which field in the dataset contains training text
    max_seq_length=max_seq_length,  # Defines the maximum sequence length for inputs
    dataset_num_proc=2,  # Uses 2 CPU threads to speed up data preprocessing

    # Define training arguments
    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples processed per device (GPU) at a time
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 steps before updating weights
        num_train_epochs=1, # Full fine-tuning run
        warmup_steps=5,  # Gradually increases learning rate for the first 5 steps
        max_steps=60,  # Limits training to 60 steps (useful for debugging; increase for full fine-tuning)
        learning_rate=2e-4,  # Learning rate for weight updates (tuned for LoRA fine-tuning)
        fp16=not is_bfloat16_supported(),  # Use FP16 (if BF16 is not supported) to speed up training
        bf16=is_bfloat16_supported(),  # Use BF16 if supported (better numerical stability on newer GPUs)
        logging_steps=10,  # Logs training progress every 10 steps
        optim="adamw_8bit",  # Uses memory-efficient AdamW optimizer in 8-bit mode
        weight_decay=0.01,  # Regularization to prevent overfitting
        lr_scheduler_type="linear",  # Uses a linear learning rate schedule
        seed=3407,  # Sets a fixed seed for reproducibility
        output_dir="outputs",  # Directory where fine-tuned model checkpoints will be saved
    ),
)


Map (num_proc=2):   0%|          | 0/30 [00:00<?, ? examples/s]

In [15]:
# Start the fine-tuning process
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30 | Num Epochs = 20
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,2.641200
20,0.622000
30,0.219500
40,0.053200
50,0.033300
60,0.027500


In [16]:
# Save the fine-tuned model
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:         train/epoch ▁▂▄▅▇██
wandb:   train/global_step ▁▂▄▅▇██
wandb:     train/grad_norm █▃▂▃▁▁
wandb: train/learning_rate █▇▅▄▂▁
wandb:          train/loss █▃▂▁▁▁
wandb: 
wandb: Run summary:
wandb:               total_flos 5288482642919424.0
wandb:              train/epoch 19.8
wandb:        train/global_step 60
wandb:          train/grad_norm 0.14498
wandb:      train/learning_rate 0
wandb:               train/loss 0.0275
wandb:               train_loss 0.59944
wandb:            train_runtime 588.0933
wandb: train_samples_per_second 0.816
wandb:   train_steps_per_second 0.102
wandb: 
wandb: 🚀 View run deft-hill-2 at: https://wandb.ai/assia-chafii93-insa-lyon/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%20Industrial%20assets%20maintenance/runs/b0gix6si
wandb: ⭐️ View project at: https://wandb.ai/assia-chafii93-insa-lyon/Fine-tune-DeepSeek-R1-Distill-Llama-8B%20on%2

## Step 5 — Run model inference after fine-tuning

In [17]:
question = """A centrifugal pump motor is drawing 15% higher current than rated, vibrating excessively, emitting a burning smell, and running 10°C hotter than normal. The discharge pressure has also dropped by 10%. What could be the root cause, and what steps should be taken to resolve the issue?"""

# Load the inference model using FastLanguageModel (Unsloth optimizes for speed)
FastLanguageModel.for_inference(model_lora)  # Unsloth has 2x faster inference!

# Tokenize the input question with a specific prompt format and move it to the GPU
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# Generate a response using LoRA fine-tuned model with specific parameters
outputs = model_lora.generate(
    input_ids=inputs.input_ids,          # Tokenized input IDs
    attention_mask=inputs.attention_mask, # Attention mask for padding handling
    max_new_tokens=1200,                  # Maximum length for generated response
    use_cache=True,                        # Enable cache for efficient generation
)

# Decode the generated response from tokenized format to readable text
response = tokenizer.batch_decode(outputs)

# Extract and print only the model's response part after "### Response:"
print(response[0].split("### Response:")[1])


<think>
A significant increase in motor load (e.g., blockage or system resistance reduction) is likely the primary cause. To address this, perform the following steps: (1) Verify for blockages or flow restrictions; (2) Monitor discharge pressure and flow rate; (3) Test for excessive load—check for system resistance changes; (4) Inspect cooling systems (e.g., clogged ventilation or fan failure); (5) Perform a thermal check to ensure overheating is not due to excessive load or insulation failure.
</think>
Verify for blockages or flow restrictions, monitor system load, and inspect cooling systems.<｜end▁of▁sentence｜>
